
<i style= 'color: green; font-size:1.5em'> Pamela Jaramillo
<!---line--->
<i style= 'color: green; font-size:1em'> Práctica 4

# Análisis de sentimientos con reviews de productos de Amazon España (opcional)

Si has hecho ya el ejercicio de web scraping con `Requests` y `BeautifulSoup` habrás visto cómo extraer datos de una página web.

El dataset que utilizarás en este ejercicio (que no es obligatorio entregar) lo he generado utilizando `Scrapy` y `BeautifulSoup`, y contiene unas $700.000$ entradas con dos columnas: el número de estrellas dadas por un usuario a un determinado producto y el comentario sobre dicho producto; exactamente igual que en el ejercico de scraping.

Ahora, tu objetivo es utilizar técnicas de procesamiento de lenguaje natural para hacer un clasificador que sea capaz de distinguir (¡y predecir!) si un comentario es positivo o negativo.

Es un ejercicio MUY complicado, más que nada porque versa sobre técnicas que no hemos visto en clase. Así que si quieres resolverlo, te va a tocar estudiar y *buscar por tu cuenta*; exactamente igual que como sería en un puesto de trabajo. Dicho esto, daré un par de pistas:

+ El número de estrellas que un usuario da a un producto es el indicador de si a dicho usuario le ha gustado el producto o no. Una persona que da 5 estrellas (el máximo) a un producto probablemente esté contento con él, y el comentario será por tanto positivo; mientras que cuando una persona da 1 estrella a un producto es porque no está satisfecha... 
+ Teniendo el número de estrellas podríamos resolver el problema como si fuera de regresión; pero vamos a establecer una regla para convertirlo en problema de clasificación: *si una review tiene 4 o más estrellas, se trata de una review positiva; y será negativa si tiene menos de 4 estrellas*. Así que probablemente te toque transformar el número de estrellas en otra variable que sea *comentario positivo/negativo*.

Y... poco más. Lo más complicado será convertir el texto de cada review en algo que un clasificador pueda utilizar y entender (puesto que los modelos no entienden de palabras, sino de números). Aquí es donde te toca investigar las técnicas para hacerlo. El ejercicio se puede conseguir hacer, y obtener buenos resultados, utilizando únicamente Numpy, pandas y Scikit-Learn; pero siéntete libre de utilizar las bibliotecas que quieras.

Ahora escribiré una serie de *keywords* que probablemente te ayuden a saber qué buscar:

`bag of words, tokenizer, tf, idf, tf-idf, sklearn.feature_extraction, scipy.sparse, NLTK (opcional), stemmer, lemmatizer, stop-words removal, bigrams, trigrams`

No te desesperes si te encuentras muy perdido/a y no consigues sacar nada. Tras la fecha de entrega os daré un ejemplo de solución explicado con todo el detalle posible.

¡Ánimo y buena suerte!

In [2]:
import pandas as pd
import numpy as np
import re
import nltk 
import matplotlib.pyplot as plt
%matplotlib inline
import pickle 

#-------REPRESENT TEXT TO NUMERICAL WORDS--------------
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

#-------TRAIN TEST SPLIT--------------

from sklearn.model_selection import train_test_split

#-------MODELOS DE CLASIFICACIÓN--------------

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import GridSearchCV as GSC
from sklearn.linear_model import SGDClassifier as SGDC
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.naive_bayes import MultinomialNB as MNB
from sklearn.svm import LinearSVC as SVC

#-------MÉTRICAS DE CLASIFICACIÓN--------------

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.preprocessing import StandardScaler
#PIPELINE
from sklearn.pipeline import Pipeline
#FEATURES SELECTION
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectKBest as SKB



In [3]:
#FUNCIONES
def metricas(modelo,x,y):
    Accuracy=(round(((modelo.score(x,y))*100),2))
    Recall=round(((recall_score(y_pred=modelo.predict(x),y_true =y))*100),2)
    precision=round(((precision_score(y_pred=modelo.predict(x),y_true =y))*100),2)
    print('Accuracy:%s'%Accuracy)
    print('   Acierto un %s%% de veces'%(Accuracy))
    print('Recall:%s'%Recall)
    print("   Capto el %s%% de comentarios positivos" %(Recall))
    print('Precisión:%s'%precision)
    print("   Cuando predigo un comentario positivo lo hago con un %s%% de fiabilidad" %(precision))      
    print('\n')
    print('Matriz de confusión')
    confusion=confusion_matrix(y_true=y, y_pred=modelo.predict(x))
    return confusion
    print('Predigo %s veces que es un comentario positivo cuando es negativo y predigo %s veces que es negativo cuando es positivo' %(confusion[0,1],
                                                                                                       confusion[1,0]))

In [2]:
df_amazon= pd.read_csv('amazon_es_reviews/amazon_es_reviews.csv', sep=';', header=0,encoding='ISO-8859-1')

In [3]:
df_amazon.head(20)

,comentario,estrellas
0,"Para chicas es perfecto, ya que la esfera no e...",4.0
1,Muy floja la cuerda y el anclaje es de mala ca...,1.0
2,"Razonablemente bien escrito, bien ambientado, ...",3.0
3,Hola! No suel o escribir muchas opiniones sobr...,5.0
4,A simple vista m parecia una buena camara pero...,1.0
5,"NI para pasar el rato, los personajes no tiene...",1.0
6,el fabricante decia que es compatible con la d...,2.0
7,"el libro estÃ¡ en muy buenas condiciones, pero...",3.0
8,"buen aspecto, pero le falta fortaleza. util pa...",3.0
9,Explica de forma simple y sencilla los pensami...,5.0


<a style='color:blue'> Creación de una nueva variable, según número de estrellas

In [4]:
valoracion=[]
for x in df_amazon['estrellas']:
    if x >=4:
        n=1
    else:
        n=0
        
    valoracion.append(n)
#1: comentarios positivos
#0: comentarios negativos

In [5]:
df_amazon['valoracion']=valoracion

In [6]:
df_amazon

,comentario,estrellas,valoracion
0,"Para chicas es perfecto, ya que la esfera no e...",4.0,1
1,Muy floja la cuerda y el anclaje es de mala ca...,1.0,0
2,"Razonablemente bien escrito, bien ambientado, ...",3.0,0
3,Hola! No suel o escribir muchas opiniones sobr...,5.0,1
4,A simple vista m parecia una buena camara pero...,1.0,0
5,"NI para pasar el rato, los personajes no tiene...",1.0,0
6,el fabricante decia que es compatible con la d...,2.0,0
7,"el libro estÃ¡ en muy buenas condiciones, pero...",3.0,0
8,"buen aspecto, pero le falta fortaleza. util pa...",3.0,0
9,Explica de forma simple y sencilla los pensami...,5.0,1


In [5]:
features = df_amazon.iloc[:, 0].values
target = df_amazon.iloc[:, 2].values

In [4]:
len(features)

702446

In [6]:
df_amazon.head()

,comentario,estrellas,valoracion,comentarios_limpios
0,"Para chicas es perfecto, ya que la esfera no e...",4.0,1,para chicas es perfecto ya que la esfera no es...
1,Muy floja la cuerda y el anclaje es de mala ca...,1.0,0,muy floja la cuerda y el anclaje es de mala ca...
2,"Razonablemente bien escrito, bien ambientado, ...",3.0,0,razonablemente bien escrito bien ambientado qu...
3,Hola! No suel o escribir muchas opiniones sobr...,5.0,1,hola no suel o escribir muchas opiniones sobre...
4,A simple vista m parecia una buena camara pero...,1.0,0,a simple vista m parecia una buena camara pero...


<a style='color:blue'> Limpieza de comentarios

In [9]:
#Limpiamos los comentarios quitando caracteres que puedan afectar a la predicción
features_limpias2=[]
for sentence in range(0, len(features)):
    features_limpias = re.sub('Ã¡','a', str(features[sentence])) #sustituir caracteres (vocales con tilde) por vocales 
    features_limpias= re.sub('Ã©','e', features_limpias)
    features_limpias= re.sub('Ã³','o', features_limpias)
    features_limpias= re.sub('Ãº','u', features_limpias)
    features_limpias= re.sub('Ã±','ñ', features_limpias)#sustituir caracteres por ñ 
    features_limpias= re.sub('\W',' ', features_limpias)#sustituir caracteres especiales por espacios en blanco 
    features_limpias= re.sub('Ã ','i', features_limpias)#sustituir caracteres (vocales con tilde) por vocales
    features_limpias = re.sub(r'\s+', ' ', features_limpias, flags=re.I)#sustituir doble espacio por un espacio 
    features_limpias = features_limpias.lower() #convertir todo a minusculas
    features_limpias2.append(features_limpias)




In [20]:
len(features_limpias2)

702446

In [19]:
features_limpias2

['para chicas es perfecto ya que la esfera no es muy grande y la correa se adapta a las muñecas mas finas un pelin gordo para mi gusto la carga por movimiento no dura mucho despues de 1 2 dias sin llevarlo se para ',
 'muy floja la cuerda y el anclaje es de mala calidad el metal se dobla muy facilmente no lo recomiendo',
 'razonablemente bien escrito bien ambientado quizas previsible pero historia interesante los personajes estan muy bien dibujados me parece un libro recomendable ',
 'hola no suel o escribir muchas opiniones sobre los productos que compro pero la verdad este producto lo merece yo estaba desencantado con las maquinas de afeitar porque hace años me compre una philips con cuchillas rotatorias yo tengo la piel extremadamente sensible y al usar esa afeitadora me hice polvo la cara vamos en carne viva me la deje la volvi a usar otra vez mas y fue mas de lo mismo a parte de no apurar bien la barba asi que despues de esta mala experiencia me pase a las cuchillas gillette mach3

In [13]:
df_amazon['comentarios_limpios']=features_limpias2

In [16]:
df_amazon.to_pickle('amazon.pickle')

In [4]:
df_amazon = pd.read_pickle('amazon.pickle')

In [7]:
df_amazon.head()

,comentario,estrellas,valoracion,comentarios_limpios
0,"Para chicas es perfecto, ya que la esfera no e...",4.0,1,para chicas es perfecto ya que la esfera no es...
1,Muy floja la cuerda y el anclaje es de mala ca...,1.0,0,muy floja la cuerda y el anclaje es de mala ca...
2,"Razonablemente bien escrito, bien ambientado, ...",3.0,0,razonablemente bien escrito bien ambientado qu...
3,Hola! No suel o escribir muchas opiniones sobr...,5.0,1,hola no suel o escribir muchas opiniones sobre...
4,A simple vista m parecia una buena camara pero...,1.0,0,a simple vista m parecia una buena camara pero...


In [6]:
features2 = df_amazon.iloc[:, 3].values
features2

array(['para chicas es perfecto ya que la esfera no es muy grande y la correa se adapta a las muñecas mas finas un pelin gordo para mi gusto la carga por movimiento no dura mucho despues de 1 2 dias sin llevarlo se para ',
       'muy floja la cuerda y el anclaje es de mala calidad el metal se dobla muy facilmente no lo recomiendo',
       'razonablemente bien escrito bien ambientado quizas previsible pero historia interesante los personajes estan muy bien dibujados me parece un libro recomendable ',
       ...,
       'si es un simple cargador muy bien de precio y funciona correctamente la verdad he de decir que me tenia un poco asustado pues tardo mucho pero lo importante es que al fin llego que era lo fundamental i entiendo la tardanza al venir desde tan lejos jijiji',
       'si el producto no esta mal lo unico que no esta claro es en que posicion debe estar el boton para tener encendido el microfono',
       'creo que es un aparato excelente y cumple todas mis expectativas aunque 

<a style='color:blue'> CONJUNTO TRAIN Y TEST

In [7]:
X_train, X_test, y_train, y_test = train_test_split(features2, target, test_size=0.3, random_state=1987)

In [10]:
print(X_train)
print(y_train)

['los raviolis que hace son tan pequeño que apenas permite el relleno ademas dificil de usar es mas util para hacer pequeñas onzas de chocolate '
 'aunque tiene muy buena pinta y parece muy robusto la verdad es que la pieza que se agarra a la ventilacion es demasiado fragil se queda en su sitio con un mecanismo de deslizar cuando intente ponerlo en el coche esa pieza ha saltado afuera y cayo dentro de la ventilacion ahora ni siquiera puedo retornarlo '
 'era para un regalo el embalaje vino chafado y la verdad que me parece caro para un juguete pero era lo que queria la niña pagas la marca seguro que que hay cosas mejores en que gastar el dinero '
 ...
 'el mejor regalo para un niño de 8 años y tambien para no tan niños es un excelente libro que combina fisiologia y anatomia de manera bastante atractiva mucho mejor que otros que he investigado y o comprado '
 'el libro es pequeño pero el contenido es adecuado y esta bien estructurado tampoco podria esperar mas por ese precio tan reducid

<a style='color:blue'> Transformación de texto en números

In [8]:
#Stopwords limpia el contenido borrando palabras que no son útiles para la predicción
stopwords1 = stopwords.words('spanish')

In [14]:
print(len(stopwords1))

313


In [17]:
#Aquí vemos las palabras que ha borrado para que no estorben en la predicción
stopwords1

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [9]:
#Transformamos los comentarios en un vector numérico. Entrenamos en train para luego transformar en train y test
vector = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords1)
X_train = vector.fit_transform(X_train).toarray()

<a style='color:green'> Definimos que el maximo de features que elija para la matriz sea de 2500, lo que significa que suara solo las 2500 palabras mas frecuentes de todos los comentarios para crear la bolsa de palabras del vector. Las palabras que son menos frecuentes no son tan útiles para la clasificación.
Max_df especifica que solo usará aquellas palabras que ocurran máximo en el 80% de los comentarios, asi mismo las palabras que aparecen en el 100% de los comentarios suelen ser comunes y tampoco aportan a la clasificación.
Min_df le indica que debe escoger palabras que al menos ocurran en 7 comentarios.

In [19]:
len(X_train[1])#2500 features seleccionadas

2500

In [27]:
X_train[1].max()

0.47350511296713044

In [10]:
#Transformamos también el conjunto de test
X_test=vector.transform(X_test).toarray()

In [47]:
vector.get_params

<bound method BaseEstimator.get_params of TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=2500, min_df=7,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde'...tuviésemos', 'tuvieseis', 'tuviesen', 'teniendo', 'tenido', 'tenida', 'tenidos', 'tenidas', 'tened'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)>

<a style='color:blue'> ESTANDARIZAMOS: Normalizando los datos

In [12]:
scaler=StandardScaler()

In [13]:
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [14]:
X_train_sc=scaler.transform(X_train)

In [16]:
X_test_sc=scaler.transform(X_test)

<a style='color:blue'> MODELOS

<a style='color:blue'> MODELO 1: RANDOM FOREST

In [11]:
gs_rfc=GSC(estimator=RFC(),
          param_grid={'n_estimators':[200]},
          cv=3,
          verbose=5)

In [12]:
gs_rfc.fit(X_train,y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] n_estimators=200 ................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...... n_estimators=200, score=0.7872230085903944, total=114.3min
[CV] n_estimators=200 ................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 115.8min remaining:    0.0s


[CV] ...... n_estimators=200, score=0.7902369679812573, total=113.3min
[CV] n_estimators=200 ................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 230.5min remaining:    0.0s


[CV] ...... n_estimators=200, score=0.7873999414291293, total=114.0min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 345.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 345.9min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [200]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=5)

In [13]:
amazon_rfc=gs_rfc.best_estimator_

In [73]:
with open("amazon_rfc.model", "wb") as archivo_salida:
    pickle.dump(amazon_rfc, archivo_salida)

In [74]:
with open("amazon_rfc.model", "rb") as archivo_entrada:
    amazon_rfc = pickle.load(archivo_entrada)

In [27]:
metricas(model_amazon_rfc,X_test,y_test)

Accuracy:78.89
   Acierto un 78.89% de veces
Recall:82.28
   Capto el 82.28% de comentarios positivos
Precisión:78.09
   Cuando predigo un comentario positivo lo hago con un 78.09% de fiabilidad


Matriz de confusión


array([[76558, 25162],
       [19322, 89692]], dtype=int64)

In [28]:
print(classification_report(y_pred=model_amazon_rfc.predict(X_test),y_true =y_test))


              precision    recall  f1-score   support

           0       0.80      0.75      0.77    101720
           1       0.78      0.82      0.80    109014

   micro avg       0.79      0.79      0.79    210734
   macro avg       0.79      0.79      0.79    210734
weighted avg       0.79      0.79      0.79    210734



In [29]:
model_amazon_rfc.score(X_train,y_train)

0.9994081901600937

<a style= 'color:green'>Se ha creado un Grid search para hacer conjuntos de validación y reducir el sobreajuste, aunque no se ha probado con distintos hiperparámetros porque da un fallo de memoria.
Vemos aun asi que el modelo sobreajusta bastante dando un 99% en train y un 79% en test

<a style='color:blue'> MODELO 2: LINEALES CON DESCENSO POR GRADIENTE ESTOCÁSTICO

In [12]:
gs_sgdc=GSC(estimator=SGDC(),
          param_grid={'penalty':['l1','l2']},
          cv=3,
          verbose=5)

<a style='color:green'> Este modelo implementa modelos de regresión lineal estimando con descenso de gradiente estocastico, la perdida del gradiente se calcula en cada batch. El modelo que lo entrena es una support vector machine.

In [51]:
gs_sgdc.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] penalty=l1 ......................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\pamej\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


[CV] ............. penalty=l1, score=0.7863505466614603, total= 1.0min
[CV] penalty=l1 ......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s
C:\Users\pamej\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


[CV] ............. penalty=l1, score=0.7873511323701678, total=  50.8s


...

[CV] ............. penalty=l2, score=0.8024148281921124, total=  43.5s
[CV] penalty=l2 ......................................................


C:\Users\pamej\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


[CV] ............. penalty=l2, score=0.7995594982428739, total=  41.7s


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  5.0min finished
C:\Users\pamej\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'penalty': ['l1', 'l2']}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=5)

In [52]:
amazon_sgdc=gs_sgdc.best_estimator_

In [53]:
amazon_sgdc

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [20]:
amazon_sgdc=SGDC()

In [71]:
with open("amazon_sgdc.model", "wb") as archivo_salida:
    pickle.dump(amazon_sgdc, archivo_salida)

In [72]:
with open("amazon_sgdc.model", "rb") as archivo_entrada:
    amazon_sgdc = pickle.load(archivo_entrada)

In [56]:
metricas(model_amazon_sgdc,X_test,y_test)

Accuracy:79.97
   Acierto un 79.97% de veces
Recall:80.28
   Capto el 80.28% de comentarios positivos
Precisión:80.86
   Cuando predigo un comentario positivo lo hago con un 80.86% de fiabilidad


Matriz de confusión


array([[81005, 20715],
       [21494, 87520]], dtype=int64)

In [57]:
model_amazon_sgdc.score(X_train,y_train)

0.8025205811531954

In [58]:
print(classification_report(y_pred=model_amazon_sgdc.predict(X_test),y_true =y_test))


              precision    recall  f1-score   support

           0       0.79      0.80      0.79    101720
           1       0.81      0.80      0.81    109014

   micro avg       0.80      0.80      0.80    210734
   macro avg       0.80      0.80      0.80    210734
weighted avg       0.80      0.80      0.80    210734



<a style= 'color:green'> Este modelo mejora los resultados del Random Forest, con un 80% de accuracy, además no está sobreajustando, (80,2% en train, 79,97% en test),  lo cual nos garantiza que va a generalizar mejor a la hora de predecir.

Además el tiempo de cómputo de este modelo es mucho más rapido que el del Random Forest

<a style='color:blue'> MODELO 3: LINEAR con SGD con datos Estandarizados

<a style='color:green'> Según la teoría para mejores resultados se debe trabajar con datos normalizados, a continuación probaremos el modelo normalizando los datos, y ya que el tiempo de cómputo no nos satura la memoria haremos un grid con más modelos en la función de pérdida. Se probarán: hinge= SVM, squearde_hinge= SVM cuadráticas, función del perceptron, SVM con squared epsilon

In [27]:
gs_sgdc3=GSC(estimator=SGDC(),
          param_grid={'penalty':['l1','l2'],
                     'loss':['hinge','squared_hinge','perceptron','squared_epsilon_insensitive']},
          cv=5,
          verbose=5)

In [28]:
gs_sgdc3.fit(X_train_sc,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] loss=hinge, penalty=l1 ..........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\pamej\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


[CV] . loss=hinge, penalty=l1, score=0.7549418368177011, total= 2.0min
[CV] loss=hinge, penalty=l1 ..........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.1min remaining:    0.0s
C:\Users\pamej\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


[CV] . loss=hinge, penalty=l1, score=0.7590347969331517, total= 1.6min
[CV] loss=hinge, penalty=l1 ..........................................


...

[CV]  loss=squared_epsilon_insensitive, penalty=l2, score=0.49031949726464785, total= 1.5min
[CV] loss=squared_epsilon_insensitive, penalty=l2 ....................


C:\Users\pamej\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


[CV]  loss=squared_epsilon_insensitive, penalty=l2, score=0.5191779707551198, total= 1.3min


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 56.6min finished
C:\Users\pamej\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'penalty': ['l1', 'l2'], 'loss': ['hinge', 'squared_hinge', 'perceptron', 'squared_epsilon_insensitive']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [29]:
amazon_sgdc3=gs_sgdc3.best_estimator_

In [30]:
amazon_sgdc3

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l1',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

<a style='color:green'> La función de pérdida que mejor funciona es SVM

In [31]:
metricas(amazon_sgdc3,X_test_sc,y_test)

Accuracy:75.43
   Acierto un 75.43% de veces
Recall:74.98
   Capto el 74.98% de comentarios positivos
Precisión:76.94
   Cuando predigo un comentario positivo lo hago con un 76.94% de fiabilidad


Matriz de confusión


array([[77217, 24503],
       [27274, 81740]], dtype=int64)

<a style='color:green'> Pero vemos que con datos estandarizados bajan las métricas del modelo

<a style='color:blue'> MODELO 4: MLP CLASSIFIER

In [30]:
gs_mlp=GSC(estimator=MLP(),
          param_grid={'activation':['relu'],
                     'alpha':[0.0001,0.001]},
          cv=3,
          verbose=5)

In [31]:
gs_mlp.fit(X_train,y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] activation=relu, alpha=0.0001 ...................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  activation=relu, alpha=0.0001, score=0.7651735162046076, total=48.0min
[CV] activation=relu, alpha=0.0001 ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 48.3min remaining:    0.0s


[CV]  activation=relu, alpha=0.0001, score=0.7634346934791097, total=38.9min
[CV] activation=relu, alpha=0.0001 ...................................


...

[CV]  activation=relu, alpha=0.001, score=0.7634896036704413, total=75.0min
[CV] activation=relu, alpha=0.001 ....................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 211.7min remaining:    0.0s


[CV]  activation=relu, alpha=0.001, score=0.7622205681374463, total=78.4min
[CV] activation=relu, alpha=0.001 ....................................
[CV]  activation=relu, alpha=0.001, score=0.7622510737992971, total=112.3min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 402.8min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'activation': ['relu'], 'alpha': [0.0001, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [32]:
amazon_mlp=gs_mlp.best_estimator_

In [68]:
with open("amazon_mlp.model", "wb") as archivo_salida:
    pickle.dump(amazon_mlp, archivo_salida)

In [69]:
with open("amazon_mlp.model", "rb") as archivo_entrada:
    amazon_mlp = pickle.load(archivo_entrada)

In [37]:
model_amazon_mlp

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [36]:
metricas(model_amazon_mlp,X_test,y_test)

Accuracy:75.57
   Acierto un 75.57% de veces
Recall:76.14
   Capto el 76.14% de comentarios positivos
Precisión:76.52
   Cuando predigo un comentario positivo lo hago con un 76.52% de fiabilidad


Matriz de confusión


array([[76246, 25474],
       [26011, 83003]], dtype=int64)

In [49]:
print(classification_report(y_pred=model_amazon_mlp.predict(X_test),y_true =y_test))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75    101720
           1       0.77      0.76      0.76    109014

   micro avg       0.76      0.76      0.76    210734
   macro avg       0.76      0.76      0.76    210734
weighted avg       0.76      0.76      0.76    210734



In [59]:
model_amazon_mlp.score(X_train,y_train)

0.9980781433033971

<a style='color:green'>La accuracy de la red neuronal en train es del 99% mientras que en test del 75%. El modelo está sobreajustando. 

<a style='color:blue'> MODELO 4: NAIVE BAYES

Naive Bayes classifier for multinomial models
The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.

In [30]:
gs_mnb2=GSC(estimator=MNB(),
          param_grid={'fit_prior':[True, False]
                     },
          cv=10,
          verbose=5)

In [31]:
gs_mnb2.fit(X_train,y_train)

Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] fit_prior=True ..................................................
[CV] ......... fit_prior=True, score=0.7826449474305005, total= 1.2min
[CV] fit_prior=True ..................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s


[CV] ......... fit_prior=True, score=0.7834699314636676, total=  40.0s
[CV] fit_prior=True ..................................................


...

[CV] ......... fit_prior=True, score=0.7845071281853124, total=  42.9s
[CV] fit_prior=True ..................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  3.5min remaining:    0.0s


[CV] ......... fit_prior=True, score=0.7830225132700169, total=  36.1s
[CV] fit_prior=True ..................................................
[CV] ......... fit_prior=True, score=0.7865408472473613, total=  39.6s
[CV] fit_prior=True ..................................................
[CV] ......... fit_prior=True, score=0.7877204043033496, total=  45.1s
[CV] fit_prior=True ..................................................
[CV] ......... fit_prior=True, score=0.7803990156799739, total=  56.2s
[CV] fit_prior=True ..................................................
[CV] ......... fit_prior=True, score=0.7846494885196559, total=  54.0s
[CV] fit_prior=True ..................................................
[CV] ......... fit_prior=True, score=0.7830225132700169, total=  44.0s
[CV] fit_prior=False .................................................
[CV] ........ fit_prior=False, score=0.7830720110629817, total=  39.5s
[CV] fit_prior=False .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 18.8min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'fit_prior': [True, False]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=5)

In [39]:
amazon_mnb2=gs_mnb2.best_estimator_

In [43]:
amazon_mnb2

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False)

In [41]:
with open("amazon_mnb.model", "wb") as archivo_salida:
    pickle.dump(amazon_mnb2, archivo_salida)

In [42]:
with open("amazon_mnb.model", "rb") as archivo_entrada:
    amazon_mnb = pickle.load(archivo_entrada)

In [44]:
metricas(amazon_mnb,X_test,y_test)

Accuracy:78.38
   Acierto un 78.38% de veces
Recall:79.69
   Capto el 79.69% de comentarios positivos
Precisión:78.77
   Cuando predigo un comentario positivo lo hago con un 78.77% de fiabilidad


Matriz de confusión


array([[78303, 23417],
       [22138, 86876]], dtype=int64)

In [45]:
amazon_mnb.score(X_train,y_train)

0.7858970291552779

<a style='color:green'> Est modelo tampoco sobreajusta. Pero sigue siendo mejor el linear con descenso de gradiente estocástico y función de pérdida SVM

<a style='color:blue'> MODELO 4: LINEAR SUPPORT VECTOR MACHINE

<a style='color:green'> Las SVM también funcionan muy bien para clasificar textos

In [18]:
gs_svc=GSC(estimator=SVC(),
          param_grid={'loss':['hinge','squared_hinge'],
                     'penalty':['l2']},
          cv=3,
          verbose=5)

In [59]:
gs_svc.fit(X_train,y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] loss=hinge, penalty=l2 ..........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\pamej\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[CV] . loss=hinge, penalty=l2, score=0.8023721202655213, total= 1.3min
[CV] loss=hinge, penalty=l2 ..........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s
C:\Users\pamej\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[CV] . loss=hinge, penalty=l2, score=0.8053555739945334, total=  49.1s


...

[CV]  loss=squared_hinge, penalty=l2, score=0.8024575361187036, total=  50.5s
[CV] loss=squared_hinge, penalty=l2 ..................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  4.0min remaining:    0.0s


[CV]  loss=squared_hinge, penalty=l2, score=0.8050322139789144, total=  52.6s
[CV] loss=squared_hinge, penalty=l2 ..................................
[CV]  loss=squared_hinge, penalty=l2, score=0.8022134908238969, total= 1.1min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  6.1min finished
C:\Users\pamej\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'loss': ['hinge', 'squared_hinge'], 'penalty': ['l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [60]:
amazon_svm=gs_svc.best_estimator_

In [61]:
with open ('amazon_svm.model','wb') as archivo_salida:
    pickle.dump(amazon_svm,archivo_salida)

In [32]:
with open ('amazon_svm.model','rb') as archivo_entrada:
    amazon_svm=pickle.load(archivo_entrada)

In [33]:
metricas(amazon_svm,X_test,y_test)

Accuracy:80.37
   Acierto un 80.37% de veces
Recall:81.51
   Capto el 81.51% de comentarios positivos
Precisión:80.73
   Cuando predigo un comentario positivo lo hago con un 80.73% de fiabilidad


Matriz de confusión


array([[80510, 21210],
       [20154, 88860]], dtype=int64)

In [34]:
amazon_svm.score(X_train,y_train)

0.8078407685799818

<a style='color:green'>Efectivamente la SVM es el modelo con mejores métricas entre todos los modelos probados, este modelo tampoco sobreajusta (80.78 en train y 80.37 en test). Garantizando que sabe generalizar en las predicciones.

<a style='color:blue'> CONCLUSIONES

<a style='color:green'> El modelo más robusto que hemos probado para este caso es es una SVM Lineal. Este modelo ha sido el que mejor accuracy nos ha dado con un 80,37%. 

Otro modelo recomendado para predicción de texto son las SVM no lineales, pero estos modelos internamente hacen multiplicaciones matriciales que para este caso al tener mas de 700k comentarios con 2500 features, se volvían muy grandes y daban un fallo de memoria.

Se intentó hacer pipelines para que vectorizen, estandarizen y entrenen los modelos, pero la memoria de computo fallaba.
